<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/GPT/gpt_legal_simplification_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 431, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 431 (delta 83), reused 152 (delta 79), pack-reused 265
Receiving objects: 100% (431/431), 243.11 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [2]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [3]:
import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [4]:
df.head()

,Название документа,Ссылка,Текст,Комментарий РГ,Date
0,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации\n Конституционный...,"Бессрочный запрет на профессию для адвокатов, ...",2022-11-25
1,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,2022-11-25
2,Федеральный закон от 21 ноября 2022 г. N 455-Ф...,https://rg.ru/documents/2022/11/24/document-po...,Принят Государственной Думой 15 ноября 2022 го...,Единое пособие для беременных женщин и граждан...,2022-11-24
3,Федеральный закон от 21 ноября 2022 г. N 443-Ф...,https://rg.ru/documents/2022/11/24/document-na...,Принят Государственной Думой 10 ноября 2022 го...,На сладкую газировку с 1 июля 2023 года введут...,2022-11-24
4,Федеральный закон от 21 ноября 2022 г. N 448-Ф...,https://rg.ru/documents/2022/11/24/document-by...,Принят Государственной Думой 10 ноября 2022 го...,Приостановленное в этом году бюджетное правило...,2022-11-24


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(df, random_state=42, test_size=0.05)

In [7]:
!pip install sacremoses
!pip install sacrebleu
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=ae10368d1b1f295b00dc85c5de10e69186bf19860b39ee08c0128d224205edad
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
!nvidia-smi

Tue Mar 21 14:33:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
train = train.drop(['Название документа', "Ссылка", 'Date'], axis=1)
test = test.drop(['Название документа', "Ссылка", 'Date'], axis=1)

In [11]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [12]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [14]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.5 MB/s eta 0:00:00


In [15]:
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from datasets import Dataset



def tokenize_function(examples):
    law_tokens = tokenizer(examples["text"], truncation=True, max_length=512, padding='max_length', return_special_tokens_mask=True)
    special_tok = tokenizer(" [SEP] ", return_special_tokens_mask=True)
    commentary_tokens = tokenizer(examples["commentary"], truncation=True, max_length=512, padding='max_length', return_special_tokens_mask=True)
    tokens = {key: torch.cat((torch.tensor(law_tokens[key]), torch.tensor(special_tok[key]), torch.tensor(commentary_tokens[key])), dim=-1) for key in law_tokens}
    return tokens

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
# Tokenize the train and validation datasets
ds = train_dataset.map(tokenize_function, remove_columns=['text', 'commentary', '__index_level_0__'])
val = test_dataset.map(tokenize_function, remove_columns=['text', 'commentary', '__index_level_0__'])

# Set up the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



Map:   0%|          | 0/1920 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [16]:
!nvidia-smi

Tue Mar 21 14:36:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
from huggingface_hub import notebook_login

notebook_login()
     

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [24]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs=30,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="logs",
    push_to_hub=True
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    eval_dataset=val,
    data_collator=data_collator,
)

# Train the model
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/content/output is already a clone of https://huggingface.co/marcus2000/output. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [20]:
from transformers import pipeline

In [21]:
test.iloc[0]["text"][:2400] + " [SEP] "

'Принят Государственной Думой 30 мая 2019 года  Одобрен Советом Федерации 11 июня 2019 года  Статья 1  Внести в Федеральный закон от 24 июля 2007 года N 221-ФЗ "О кадастровой деятельности" (Собрание законодательства Российской Федерации, 2007, N 31, ст. 4017; 2013, N 30, ст. 4083; 2014, N 52, ст. 7558; 2016, N 27, ст. 4294; 2017, N 31, ст. 4766; 2018, N 53, ст. 8464) следующие изменения:  1) статью 35 дополнить частью 11 следующего содержания:  "11. Кадастровые работы могут выполняться кадастровым инженером, являющимся работником юридического лица, на основании трудового договора в случае, если кадастровые работы выполняются для собственных нужд такого юридического лица. При этом заключение предусмотренного частью 1 настоящей статьи договора подряда не требуется.";  2) в статье 421:  а) в части 1:  пункт 2 изложить в следующей редакции:  "2) земельных участков, образование которых предусмотрено документами, указанными в части 6 настоящей статьи;";  пункт 3 изложить в следующей редакции

In [22]:
device = model.device

In [23]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer,device=0)

# Generate text
prompt = test.iloc[0]["text"][:2400] + " [SEP] "
generated_text = text_generator(prompt, max_length=1800, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:

generated_text

[{'generated_text': 'Принят Государственной Думой 30 мая 2019 года  Одобрен Советом Федерации 11 июня 2019 года  Статья 1  Внести в Федеральный закон от 24 июля 2007 года N 221-ФЗ "О кадастровой деятельности" (Собрание законодательства Российской Федерации, 2007, N 31, ст. 4017; 2013, N 30, ст. 4083; 2014, N 52, ст. 7558; 2016, N 27, ст. 4294; 2017, N 31, ст. 4766; 2018, N 53, ст. 8464) следующие изменения:  1) статью 35 дополнить частью 11 следующего содержания:  "11. Кадастровые работы могут выполняться кадастровым инженером, являющимся работником юридического лица, на основании трудового договора в случае, если кадастровые работы выполняются для собственных нужд такого юридического лица. При этом заключение предусмотренного частью 1 настоящей статьи договора подряда не требуется.";  2) в статье 421:  а) в части 1:  пункт 2 изложить в следующей редакции:  "2) земельных участков, образование которых предусмотрено документами, указанными в части 6 настоящей статьи;";  пункт 3 изложить 